In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())


cuda:1


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/MURI Aug17 Thin Plate'

In [4]:
# !pip install smt

In [4]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1]).reshape(-1,1)
b_value = np.array([0.0,0.25,0.5,1.0]).reshape(-1,1)


LR_tune, B_value = np.meshgrid(lr_tune,b_value)

LR_tune = LR_tune.flatten('F').reshape(-1,1)
B_value = B_value.flatten('F').reshape(-1,1)


lrb_tune = np.hstack((LR_tune,B_value))

In [6]:
#Material Properties This link - https://www.mathworks.com/help/pde/ug/nonlinear-heat-transfer-in-a-thin-plate.html#heatTransferThinPlateExample-1
k = 400
rho = 8960
cp = 386
t_z = 0.01
stef_bolt = 5.670373e-8
hc = 1
Ta = 300
emiss = 0.5


In [7]:
label = "Thinplate_stan"
loss_thresh = 10000
x = np.linspace(0,1,100).reshape(-1,1)
y = np.linspace(0,1,100).reshape(-1,1)
t = np.linspace(0,1,100).reshape(-1,1) #t is actually from 0 to 5000, let us scale it to 0 to 1

X,Y,T = np.meshgrid(x,y,t)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xyt = np.hstack((X,Y,T))

initial_pts = np.logical_and(T==0,Y!=0).reshape(-1,)

DBC_pts = (Y == 0).reshape(-1,)


NBC_pts_x0 = (X == 0).reshape(-1,)
NBC_pts_x1 = (X == 1).reshape(-1,)

NBC_pts_y0 = (Y == 0).reshape(-1,)
NBC_pts_y1 = (Y == 1).reshape(-1,)

xyt_initial = xyt[initial_pts,:]
xyt_DBC = xyt[DBC_pts,:]

xyt_NBC_x0 = xyt[NBC_pts_x0,:]
xyt_NBC_x1 = xyt[NBC_pts_x1,:]

#xyt_NBC_y0 = xyt[NBC_pts_y0,:]
xyt_NBC_y1 = xyt[NBC_pts_y1,:]

u_initial = 300*np.ones((np.shape(xyt_initial)[0],1))
u_DBC = 1000*np.ones((np.shape(xyt_DBC)[0],1))

xyt_I_DBC = np.vstack((xyt_initial,xyt_DBC))
#xyt_NBC = np.vstack((xyt_NBC_1,xyt_NBC_2,xyt_NBC_3,xyt_NBC_4))
xyt_NBC_x = np.vstack((xyt_NBC_x0,xyt_NBC_x1))
#xyt_NBC_y = np.vstack((xyt_NBC_y0,xyt_NBC_y1))
xyt_NBC_y = np.vstack((xyt_NBC_y1))

u_I_DBC = np.vstack((u_initial,u_DBC))


lb_xyt = xyt[0]
ub_xyt = xyt[-1]

In [8]:
fea_data = scipy.io.loadmat('./../3D_HTTP_FEA.mat')
xy = fea_data['xy']
t = fea_data['t']/3000
xyt = np.zeros((497*101,3))
u_true = np.ones((497*101,1))


for i in range(101):
    t_temp = t[0,i]*np.ones((497,1))
    xyt[497*i:497*(i+1)] = np.hstack((xy,t_temp))
    u_true[497*i:497*(i+1)] = fea_data['u'][:,i].reshape(-1,1)
    #print(i)
#print(xyt)

xyt_test_tensor = torch.from_numpy(xyt).float().to(device)
u_true_norm = np.linalg.norm(u_true,2)

In [9]:
def trainingdata(N_D,N_N,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    
    #choose random N_u points for training
    idx = np.random.choice(xyt_I_DBC.shape[0], N_D, replace=False) 
    xyt_D = xyt_I_DBC[idx,:] #choose indices from  set 'idx' (x,t)
    u_D = u_I_DBC[idx].reshape(-1,1)      #choose corresponding u

    idx = np.random.choice(xyt_NBC_x.shape[0], N_D, replace=False) 
    xyt_Nx = xyt_NBC_x[idx,:] #choose indices from  set 'idx' (x,t)

    idx = np.random.choice(xyt_NBC_y.shape[0], N_D, replace=False) 
    xyt_Ny = xyt_NBC_y[idx,:] #choose indices from  set 'idx' (x,t)

    '''Collocation Points'''
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    
    xyt_coll = lb_xyt + (ub_xyt - lb_xyt)*samples
    xyt_coll = np.vstack((xyt_coll, xyt_D,xyt_Nx,xyt_Ny)) # append training points to collocation points 

    return xyt_coll, xyt_D, u_D, xyt_Nx,xyt_Ny

In [10]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,beta_init):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
        
        self.beta = Parameter(beta_init*torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True

    
            
    'foward pass'
    def forward(self,xyt):
        if torch.is_tensor(xyt) != True:         
            xyt = torch.from_numpy(xyt)                
        
        ubxyt = torch.from_numpy(ub_xyt).float().to(device)
        lbxyt = torch.from_numpy(lb_xyt).float().to(device)
    
                      
        #preprocessing input 
        xyt = (xyt - lbxyt)/(ubxyt - lbxyt)
        
        #convert to float
        a = xyt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            z1 = self.activation(z) 
            a = z1 + self.beta[:,i]*z*z1
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_D(self,xyt_D,u_D):
                
        loss_bc = self.loss_function(self.forward(xyt_D), u_D)
                
        return loss_bc
    
    def loss_N(self,xyt_Nx,xyt_Ny,N_hat):
        
        g1 = xyt_Nx.clone()             
        g1.requires_grad = True
        u1 = self.forward(g1)
        
        u1_x_y_t = autograd.grad(u1,g1,torch.ones([xyt_Nx.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du1_dx = u1_x_y_t[:,[0]]
        
        g2 = xyt_Ny.clone()             
        g2.requires_grad = True
        u2 = self.forward(g2)
        
        u2_x_y_t = autograd.grad(u2,g2,torch.ones([xyt_Ny.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du2_dy = u2_x_y_t[:,[1]]
               
        loss_N1 = self.loss_function(du1_dx,N_hat)
        loss_N2 = self.loss_function(du2_dy,N_hat)
        
        #return loss_N1+loss_N2       
        return loss_N1 + loss_N2
    
    def loss_PDE(self, xyt_coll, f_hat):
        
        g = xyt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_y_t = autograd.grad(u,g,torch.ones([xyt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_yy_tt = autograd.grad(u_x_y_t,g,torch.ones(xyt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dt = u_x_y_t[:,[2]]
        
        d2u_dx2 = u_xx_yy_tt[:,[0]]
        d2u_dy2 = u_xx_yy_tt[:,[1]]    
        

        f = rho*cp*t_z*du_dt/3000 - k*t_z*(d2u_dx2+d2u_dy2) + 2*hc*(u-Ta) + 2*emiss*stef_bolt*(torch.pow(u,4)-Ta**4) 
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat):

        loss_D = self.loss_D(xyt_D,u_D)
        loss_N = self.loss_N(xyt_Nx,xyt_Ny,N_hat)
        loss_f = self.loss_PDE(xyt_coll,f_hat)
        
        loss_val = loss_D + loss_N + loss_f
        
        #print(self.iter,"loss_D:",loss_D.cpu().detach().numpy(),"loss_N:",loss_N.cpu().detach().numpy(),"loss_f:",loss_f.cpu().detach().numpy())
        
        return loss_val
       
    'test neural network'
    def test(self):
        u_pred = self.forward(xyt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
        
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [11]:
def train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [12]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [13]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xyt_coll_np_array, xyt_D_np_array, u_D_np_array,xyt_Nx_np_array,xyt_Ny_np_array = trainingdata(N_D,N_N,N_f,(reps)*22)

    xyt_coll = torch.from_numpy(xyt_coll_np_array).float().to(device)
    xyt_D = torch.from_numpy(xyt_D_np_array).float().to(device)
    u_D = torch.from_numpy(u_D_np_array).float().to(device)
    xyt_Nx = torch.from_numpy(xyt_Nx_np_array).float().to(device)
    xyt_Ny = torch.from_numpy(xyt_Ny_np_array).float().to(device)

    N_hat = torch.zeros(xyt_Nx.shape[0],1).to(device)    
    f_hat = torch.zeros(xyt_coll.shape[0],1).to(device)

    nan_flag = 0
    
    for i in range(max_iter):
        train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,i)

        loss_np = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])

        if(np.isnan(loss_np)):
            nan_flag =1
            print("NAN BREAK!")
            break
    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [14]:
nan_tune = []
for tune_reps in range(20):
    label = "3D_HTTP_stan_tune"+str(tune_reps)
    max_reps = 10
    max_iter = 100

    train_loss_full = []
    test_mse_full = []
    test_re_full = []
    beta_full = []
    elapsed_time= np.zeros((max_reps,1))
    
    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))
    
    beta_init = lrb_tune[tune_reps,1]
    
    for reps in range(max_reps):
        print(label)
        train_loss = []
        test_mse_loss = []
        test_re_loss = []
        beta_val = []


        print(reps)

        torch.manual_seed(reps*36)
        N_D = 5000 #Total number of data points for 'y'
        N_N = 3500
        N_f = 10000 #Total number of collocation points 

        layers = np.array([3,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
 
        PINN = Sequentialmodel(layers,beta_init)

        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())


        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lrb_tune[tune_reps,0], 
                                  max_iter = 10, 
                                  max_eval = 15, 
                                  tolerance_grad = -1, 
                                  tolerance_change = -1, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')



        nan_flag = train_model(max_iter,reps)


        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)
        test_mse_full.append(test_mse_loss)
        test_re_full.append(test_re_loss)
        #elapsed_time[reps] = time.time() - start_time
        beta_full.append(beta_val)
        
        if(nan_flag == 1):
            nan_tune.append(tune_reps)
            break

        #print('Training time: %.2f' % (elapsed_time[reps]))

    mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full, "label": label}
    savemat(label+'.mat', mdic)

3D_HTTP_stan_tune0
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 1235712.9 Test MSE 309898.57204455783 Test RE 0.9868454651639196
1 Train Loss 755159.2 Test MSE 294277.9055950005 Test RE 0.9616525563817746
2 Train Loss 640241.5 Test MSE 295495.6320661277 Test RE 0.9636401688000372
3 Train Loss 587557.25 Test MSE 287451.5633690657

96 Train Loss 33266.445 Test MSE 4182.9469918487475 Test RE 0.11465171164290115
97 Train Loss 32895.246 Test MSE 4014.139147872383 Test RE 0.11231443421686425
98 Train Loss 31824.242 Test MSE 3811.833936185768 Test RE 0.10944762662070608
99 Train Loss 31483.512 Test MSE 3672.8853241717616 Test RE 0.10743432181092802
Training time: 88.88
3D_HTTP_stan_tune0
1
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9)

91 Train Loss 20036.709 Test MSE 4281.280354897657 Test RE 0.11599150840602343
92 Train Loss 19916.111 Test MSE 4198.498428069143 Test RE 0.11486464102520207
93 Train Loss 19788.193 Test MSE 4328.421137202722 Test RE 0.1166283460546253
94 Train Loss 19725.062 Test MSE 4309.234776750082 Test RE 0.11636957286339342
95 Train Loss 19637.174 Test MSE 4205.64452957453 Test RE 0.11496235279257945
96 Train Loss 19468.879 Test MSE 4101.56866916381 Test RE 0.11353097189370868
97 Train Loss 19259.562 Test MSE 3973.453755671631 Test RE 0.11174380195929215
98 Train Loss 19152.203 Test MSE 3898.9151439224233 Test RE 0.11069073057203227
99 Train Loss 19093.783 Test MSE 3910.645107897042 Test RE 0.11085711316000994
Training time: 88.98
3D_HTTP_stan_tune0
2
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_

86 Train Loss 20412.783 Test MSE 2201.5038524676365 Test RE 0.083176202269806
87 Train Loss 20229.074 Test MSE 2081.6950568651214 Test RE 0.08088126175646512
88 Train Loss 20125.42 Test MSE 2020.2956012005639 Test RE 0.07967954057620037
89 Train Loss 20020.85 Test MSE 2013.434798193765 Test RE 0.07954413204194695
90 Train Loss 19908.47 Test MSE 1980.2152148644825 Test RE 0.0788852050468161
91 Train Loss 19829.47 Test MSE 1891.6516628584302 Test RE 0.07710098838452108
92 Train Loss 19739.75 Test MSE 1884.8104856727887 Test RE 0.076961443855827
93 Train Loss 19659.715 Test MSE 1865.1657253138947 Test RE 0.07655932139351561
94 Train Loss 19589.758 Test MSE 1830.0882505952109 Test RE 0.07583599313845953
95 Train Loss 19489.924 Test MSE 1817.3662578541155 Test RE 0.07557194371337646
96 Train Loss 19387.584 Test MSE 1777.8462748986592 Test RE 0.07474574333717326
97 Train Loss 19235.99 Test MSE 1741.0829474615925 Test RE 0.07396888856796707
98 Train Loss 19035.11 Test MSE 1861.159049968662 Te

81 Train Loss 22044.018 Test MSE 3030.61967730175 Test RE 0.09758994686850898
82 Train Loss 21783.8 Test MSE 3082.6727229397534 Test RE 0.09842446708344035
83 Train Loss 21598.703 Test MSE 3324.430995743377 Test RE 0.1022110909209811
84 Train Loss 21491.484 Test MSE 3228.302853260733 Test RE 0.10072250010841181
85 Train Loss 21429.17 Test MSE 3115.119515720372 Test RE 0.09894109648757449
86 Train Loss 21335.523 Test MSE 3099.53584690285 Test RE 0.09869330528067044
87 Train Loss 21146.67 Test MSE 3035.5063224847645 Test RE 0.09766859338524826
88 Train Loss 20971.512 Test MSE 2984.403234905472 Test RE 0.09684297295015318
89 Train Loss 20859.084 Test MSE 3065.1374877800426 Test RE 0.09814413282483644
90 Train Loss 20780.785 Test MSE 3086.521123369956 Test RE 0.09848588434262681
91 Train Loss 20618.137 Test MSE 3108.4970693997634 Test RE 0.0988358708761748
92 Train Loss 20420.135 Test MSE 3088.3084846187708 Test RE 0.09851439611517969
93 Train Loss 20314.016 Test MSE 3133.143822357032 Test

...

70 Train Loss 25600.559 Test MSE 3133.864969494247 Test RE 0.09923834253445443
71 Train Loss 25494.324 Test MSE 3113.121018892448 Test RE 0.09890935369399807
72 Train Loss 25379.434 Test MSE 3066.419788956613 Test RE 0.09816465999170372
73 Train Loss 25273.193 Test MSE 3014.04409555878 Test RE 0.09732270316742171
74 Train Loss 25165.047 Test MSE 2939.114190898351 Test RE 0.09610535605448303
75 Train Loss 25051.213 Test MSE 2910.0649749182826 Test RE 0.0956292401855897
76 Train Loss 24939.871 Test MSE 2847.3676077882387 Test RE 0.09459346458672929
77 Train Loss 24825.588 Test MSE 2859.8117937624684 Test RE 0.09479994570949699
78 Train Loss 24727.877 Test MSE 2817.4013337539086 Test RE 0.09409438761629484
79 Train Loss 24598.486 Test MSE 2831.5780733117313 Test RE 0.09433082495995747
80 Train Loss 24447.709 Test MSE 2755.5745392612025 Test RE 0.09305622784695036
81 Train Loss 24347.803 Test MSE 2771.107888925691 Test RE 0.09331814120980574
82 Train Loss 24241.09 Test MSE 2744.13811387405

65 Train Loss 24013.332 Test MSE 2684.792202876454 Test RE 0.09185328675739036
66 Train Loss 23911.68 Test MSE 2707.846535128608 Test RE 0.092246816209391
67 Train Loss 23778.775 Test MSE 2684.0599900591865 Test RE 0.09184076051047042
68 Train Loss 23662.262 Test MSE 2720.0413330981123 Test RE 0.09245429982841771
69 Train Loss 23459.76 Test MSE 2828.213172456643 Test RE 0.0942747593631461
70 Train Loss 23272.852 Test MSE 2875.5074893350156 Test RE 0.09505973816673434
71 Train Loss 23155.938 Test MSE 2794.701433887615 Test RE 0.09371456013257318
72 Train Loss 23052.64 Test MSE 2848.9781770192644 Test RE 0.0946202134662359
73 Train Loss 22919.793 Test MSE 2808.8542419130604 Test RE 0.09395155311194392
74 Train Loss 22823.271 Test MSE 2780.954962536769 Test RE 0.09348379620364937
75 Train Loss 22744.062 Test MSE 2812.5958545086623 Test RE 0.09401410767969677
76 Train Loss 22680.629 Test MSE 2717.601072269479 Test RE 0.09241281824726308
77 Train Loss 22511.914 Test MSE 2636.990373308042 Te

60 Train Loss 27229.658 Test MSE 3502.391079775668 Test RE 0.10491115812204724
61 Train Loss 26985.96 Test MSE 3523.6910645199923 Test RE 0.10522968607031027
62 Train Loss 26875.293 Test MSE 3462.6755409625675 Test RE 0.10431463957551919
63 Train Loss 26810.8 Test MSE 3378.6420747044785 Test RE 0.10304109263552064
64 Train Loss 26732.953 Test MSE 3420.3905759842187 Test RE 0.10367575644955221
65 Train Loss 26616.26 Test MSE 3274.877057922976 Test RE 0.10144645200737336
66 Train Loss 26105.809 Test MSE 3287.7449278549852 Test RE 0.10164556176373142
67 Train Loss 25861.805 Test MSE 3293.9448256442233 Test RE 0.10174135618962642
68 Train Loss 25758.148 Test MSE 3298.5686068520376 Test RE 0.10181273941299072
69 Train Loss 25609.344 Test MSE 3198.6999655886802 Test RE 0.10025963403652191
70 Train Loss 25408.367 Test MSE 3238.169977371167 Test RE 0.1008763089683314
71 Train Loss 25302.338 Test MSE 3268.527913289612 Test RE 0.10134806500335307
72 Train Loss 25075.582 Test MSE 3150.07775901561

55 Train Loss 24858.188 Test MSE 2825.494995976959 Test RE 0.09422944505710827
56 Train Loss 24724.283 Test MSE 2770.2590048155325 Test RE 0.09330384686291911
57 Train Loss 24599.611 Test MSE 2655.2641895665556 Test RE 0.09134677756620327
58 Train Loss 24382.545 Test MSE 2639.0643854169443 Test RE 0.09106769725967917
59 Train Loss 24254.875 Test MSE 2616.6279055573223 Test RE 0.09067975677107255
60 Train Loss 24196.492 Test MSE 2605.1849170650667 Test RE 0.09048126000477884
61 Train Loss 24157.9 Test MSE 2597.0253225583533 Test RE 0.09033945252947745
62 Train Loss 24019.463 Test MSE 2584.349714305741 Test RE 0.09011871763987742
63 Train Loss 23763.008 Test MSE 2521.89818315958 Test RE 0.08902318674816502
64 Train Loss 23684.734 Test MSE 2443.5692909568384 Test RE 0.08762977397228593
65 Train Loss 23551.564 Test MSE 2473.298736171401 Test RE 0.08816123188574099
66 Train Loss 23345.832 Test MSE 2415.0489808675757 Test RE 0.08711688414118023
67 Train Loss 23228.51 Test MSE 2492.1353951379

50 Train Loss 18845.143 Test MSE 2217.4155651163037 Test RE 0.08347624558440468
51 Train Loss 18666.045 Test MSE 2028.941549355541 Test RE 0.0798498546874064
52 Train Loss 18576.17 Test MSE 1948.9790973995248 Test RE 0.07826056030384543
53 Train Loss 18536.531 Test MSE 1927.7990158458438 Test RE 0.07783415937674436
54 Train Loss 18354.148 Test MSE 1953.7042355399863 Test RE 0.07835537099157454
55 Train Loss 18295.188 Test MSE 1920.5471103792336 Test RE 0.07768762496400061
56 Train Loss 18243.096 Test MSE 1885.1787488235157 Test RE 0.076968962033317
57 Train Loss 18225.898 Test MSE 1834.89319547192 Test RE 0.07593548257636862
58 Train Loss 18096.16 Test MSE 1824.6827808858197 Test RE 0.07572391321280744
59 Train Loss 17923.53 Test MSE 1838.189000047331 Test RE 0.07600364900434184
60 Train Loss 17885.365 Test MSE 1834.0953983974787 Test RE 0.07591897270743624
61 Train Loss 17752.521 Test MSE 1948.4034035353736 Test RE 0.07824900105932325
62 Train Loss 17689.484 Test MSE 1958.174714269097

In [1]:
import scipy.io as sio
import numpy as np

In [2]:
for tune_reps in range(20):
    label = "3D_HTTP_stan_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

0   0.09609899982431741
1   0.10169314864136496
2   0.09116847875522556
3   0.10093728206104721
4   0.09569225291219967
5   0.09734600145785213
6   0.08658959764251058
7   0.08117427206522203
8   [[0.98850215 0.96893064 0.95828147 0.96307055 0.94625116 0.74728444
  0.53976973 0.47954647 0.43163023 0.37057204 0.33691905 0.28710747
  0.26953237 0.26010415 0.24492778 0.22301686 0.21326281 0.20844129
  0.20067328 0.18793856 0.15671362 0.15170994 0.15498957 0.14265135
  0.14032166 0.140709   0.13100316 0.12590612 0.12588283 0.12566684
  0.12194267 0.11674532 0.11173624 0.11004516 0.10666791 0.09829153
  0.10029908 0.10195494 0.09970524 0.10033042 0.10275805 0.1049671
  0.10260797 0.10301312 0.10394498 0.10305352 0.09995045 0.09845929
  0.09693394 0.09518737 0.0924737  0.08960752 0.08868249 0.08706556
  0.08395378 0.08385178 0.08463179 0.08463513 0.08634004 0.0867429
  0.08711981 0.08693662 0.08363387 0.08265391 0.08239917 0.08275657
  0.08412701 0.08419537 0.08320796 0.08469139 0.08310656 0

In [7]:
lrb_tune[10]

array([0.25, 0.5 ])